In [1]:

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf  # Version 1.0.0 
from sklearn import metrics

import os
import timeit


model_path = "/home/soon2soon/Notebooks/DreamChallenge/LSTM-Human-Activity-Recognition/tmp/"

print("Load model from " + model_path)

Load model from /home/soon2soon/Notebooks/DreamChallenge/LSTM-Human-Activity-Recognition/tmp/


In [2]:
INPUT_SIGNAL_TYPES = [
    "body_acc_x_",
    "body_acc_y_",
    "body_acc_z_",
    "body_rotation_x_",
    "body_rotation_y_",
    "body_rotation_z_",
#     "body_attitude_x_",
#     "body_attitude_y_",
#     "body_attitude_z_",
    "body_gravity_x_",
    "body_gravity_y_",
    "body_gravity_z_"
]


LABELS = [
    "FALSE",
    "TRUE" 
] 



DATASET_PATH = "/home/soon2soon/Notebooks/DreamChallenge/data/processed_data_test/"

print("target data-->", DATASET_PATH)


# DATASET_FINAL_TEST_PATH = "/home/soon2soon/Notebooks/DreamChallenge/data/processed_data_test/"


print("\n" + "Dataset is now located at: " + DATASET_PATH)


startTime = timeit.default_timer()


TRAIN = "train/"
TEST = "test/"


def load_X(X_signals_paths):
    X_signals = []
    
    for signal_type_path in X_signals_paths:
        file = open(signal_type_path, 'r')
        print(signal_type_path)
        
        X_signals.append(
            [np.array(serie, dtype=np.float32) for serie in [
                row.replace('  ', ' ').strip().split(' ') for row in file
                
            ]]
        )
        
        file.close()

    
    return np.transpose(np.array(X_signals), (1, 2, 0))

# X_train_signals_paths = [
#     DATASET_PATH + TRAIN + signal + "train.txt" for signal in INPUT_SIGNAL_TYPES
    
# ]


X_test_signals_paths = [
    DATASET_PATH + TEST + signal + "test.txt" for signal in INPUT_SIGNAL_TYPES
]



# X_train = load_X(X_train_signals_paths)


X_test = load_X(X_test_signals_paths)

# X_test_final = load_X(X_test_final_signals_paths)



def load_y(y_path):
    file = open(y_path, 'r')
    print(y_path)
    # Read dataset from disk, dealing with text file's syntax
    y_ = np.array(
        [elem for elem in [
            row.replace('  ', ' ').strip().split(' ') for row in file
        ]], 
        dtype=np.int32
    )
    file.close()
    
    return y_

y_train_path = DATASET_PATH + TRAIN + "y_train.txt"

y_test_path = DATASET_PATH + TEST + "y_test.txt"



# y_train = load_y(y_train_path)

y_test = load_y(y_test_path)

# y_test_final = load_y(y_test_final_path)


endTime = timeit.default_timer()

print("Dataset load done.")

# Input Data 

# training_data_count = len(X_train)  # training series (with 50% overlap between each serie)


# print("+ training_data_count",training_data_count )

test_data_count = len(X_test)  # number of testing series
# n_steps = len(X_train[0])  # timesteps per series
# n_input = len(X_train[0][0])  # number of input parameters per timestep


n_steps = 512
n_input = 9



print("+ test_data_count",test_data_count )

print("+ n_steps", n_steps)
print("+ n_input", n_input)



n_hidden = 96 # Hidden layer num of features
n_classes = 2 
n_hidden_3 = 32

# Training parameters

learning_rate = 0.001
lambda_loss_amount = 0.0015
# training_iters = training_data_count * 2  # Loop xx times on the dataset
batch_size = 300
display_iter = 5000  # To show test set accuracy during training


print("(X shape, y shape, every X's mean, every X's standard deviation)")
print(X_test.shape, y_test.shape, np.mean(X_test), np.std(X_test))


print ("{:.2f} seconds to execute this".format(endTime - startTime))


target data--> /home/soon2soon/Notebooks/DreamChallenge/data/processed_data_test/

Dataset is now located at: /home/soon2soon/Notebooks/DreamChallenge/data/processed_data_test/
/home/soon2soon/Notebooks/DreamChallenge/data/processed_data_test/test/body_acc_x_test.txt
/home/soon2soon/Notebooks/DreamChallenge/data/processed_data_test/test/body_acc_y_test.txt
/home/soon2soon/Notebooks/DreamChallenge/data/processed_data_test/test/body_acc_z_test.txt
/home/soon2soon/Notebooks/DreamChallenge/data/processed_data_test/test/body_rotation_x_test.txt
/home/soon2soon/Notebooks/DreamChallenge/data/processed_data_test/test/body_rotation_y_test.txt
/home/soon2soon/Notebooks/DreamChallenge/data/processed_data_test/test/body_rotation_z_test.txt
/home/soon2soon/Notebooks/DreamChallenge/data/processed_data_test/test/body_gravity_x_test.txt
/home/soon2soon/Notebooks/DreamChallenge/data/processed_data_test/test/body_gravity_y_test.txt
/home/soon2soon/Notebooks/DreamChallenge/data/processed_data_test/test/b

In [3]:
def one_hot(y_):
    # Function to encode output labels from number indexes 
    # e.g.: [[5], [0], [3]] --> [[0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]
    
    y_ = y_.reshape(len(y_))
      
    n_values = int(np.max(y_)) + 1
    
    if n_values == 1:
        n_values = 2
    
    return np.eye(n_values)[np.array(y_, dtype=np.int32)]  # Returns FLOATS

def extract_batch_size(_train, step, batch_size):
    # Function to fetch a "batch_size" amount of data from "(X|y)_train" data. 
    
    shape = list(_train.shape)
    shape[0] = batch_size
    batch_s = np.empty(shape)

    for i in range(batch_size):
        # Loop index
        index = ((step-1)*batch_size + i) % len(_train)
        batch_s[i] = _train[index] 

    return batch_s

def LSTM_RNN(_X, _weights, _biases):
    
    # input shape: (batch_size, n_steps, n_input)
    _X = tf.transpose(_X, [1, 0, 2])  # permute n_steps and batch_size
    # Reshape to prepare input to hidden activation
    _X = tf.reshape(_X, [-1, n_input]) 
    # new shape: (n_steps*batch_size, n_input)
    
    # Linear activation
    _X = tf.nn.relu(tf.matmul(_X, _weights['hidden']) + _biases['hidden'])
    # Split data because rnn cell needs a list of inputs for the RNN inner loop
    _X = tf.split(_X, n_steps, 0) 
    # new shape: n_steps * (batch_size, n_hidden)

    # Define two stacked LSTM cells (two recurrent layers deep) with tensorflow
    lstm_cell_1 = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
    
    lstm_cell_1 = tf.contrib.rnn.DropoutWrapper(lstm_cell_1, input_keep_prob=0.7, output_keep_prob=0.7)
    
    lstm_cell_2 = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
    
    lstm_cell_2 = tf.contrib.rnn.DropoutWrapper(lstm_cell_2, input_keep_prob=0.7, output_keep_prob=0.7) 
    
        
    lstm_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell_1, lstm_cell_2], state_is_tuple=True)
    


    # Get LSTM cell output
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cells, _X, dtype=tf.float32)
    

    # Get last time step's output feature for a "many to one" style classifier
    lstm_last_output = outputs[-1]
    
    # Linear activation
    return tf.matmul(lstm_last_output, _weights['out']) + _biases['out'], lstm_last_output

# Graph input/output
x = tf.placeholder(tf.float32, [None, n_steps, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])


# Graph weights
weights = {
    'hidden': tf.Variable(tf.random_normal([n_input, n_hidden])), # Hidden layer weights
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes], mean=1.0))
}
biases = {
    'hidden': tf.Variable(tf.random_normal([n_hidden])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

weights_3 = {
    'hidden_acc': tf.Variable(tf.random_normal([3, n_hidden_3])), # Hidden layer weights
    'hidden_rot': tf.Variable(tf.random_normal([3, n_hidden_3])),
    'hidden_gra': tf.Variable(tf.random_normal([3, n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes], mean=1.0))
}
biases_3 = {
    'hidden_acc': tf.Variable(tf.random_normal([n_hidden_3])),
    'hidden_rot': tf.Variable(tf.random_normal([n_hidden_3])),
    'hidden_gra': tf.Variable(tf.random_normal([n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# pred = LSTM_RNN_3(x, weights, biases)
# pred, last_state = LSTM_RNN_3(x, weights_3, biases_3)
pred, last_state = LSTM_RNN(x, weights, biases)
# pred = BiRNN(x, weights_bi, biases)
# pred = BiRNN_attn(x, weights_bi, biases) 



# Loss, optimizer and evaluation
l2 = lambda_loss_amount * sum(
    tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables()
) # L2 loss prevents this overkill neural network to overfit the data

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred)) + l2 # Softmax loss
# optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost) # RMS Optimizer

correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [4]:
with tf.Session() as sess:
    
    sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))
    init = tf.global_variables_initializer()
    sess.run(init)
    
    new_saver = tf.train.import_meta_graph(model_path+'model.ckpt.meta')
    new_saver.restore(sess, tf.train.latest_checkpoint(model_path))
    print("model load done.")
   
    
    
    
    itter = 1
    chunk_size = 500
    while itter * chunk_size <= len(X_test):
        batch_xs =         extract_batch_size(X_test, itter, chunk_size)
        batch_ys = one_hot(extract_batch_size(y_test, itter, chunk_size))

        # Fit training using batch data

        a_t, _, l_h_t = sess.run(
            [accuracy, cost, last_state],
            feed_dict={
                x: batch_xs,
                y: batch_ys
            }
        )

        print("FINAL RESULT: " + \
              ", Accuracy = {}".format(a_t))

        ##########################
        
        np.savetxt('output_state_result_test/'+"output_hidden_test"+str(itter)+".txt", l_h_t)
        
        ##########################
        
        print(itter, "hidden test final write donw!")
        itter= itter+1

    print("iter done.")
    
    
    
#     one_hot_predictions, accuracy, final_loss, l_h = sess.run(
#         [pred, accuracy, cost, last_state],
#         feed_dict={
#             x: X_test,
#             y: one_hot(y_test)
#         }
#     )


#     print("FINAL RESULT: " + \
#           "Batch Loss = {}".format(final_loss) + \
#           ", Accuracy = {}".format(accuracy))


#     np.savetxt("output_hidden_train.txt", l_h)

#     print("hidden test write donw")
    
    
    sess.close()

model load done.
FINAL RESULT: , Accuracy = 0.4919999837875366
1 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.47200000286102295
2 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.48399996757507324
3 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.45399996638298035
4 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.48799997568130493
5 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.4699999988079071
6 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.4700000286102295
7 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.5160000324249268
8 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.4359999895095825
9 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.44200003147125244
10 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.44200000166893005
11 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.550000011920929
12 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.5099999904

105 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.4440000355243683
106 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.48399999737739563
107 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.4959999918937683
108 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.4480000436306
109 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.5339999794960022
110 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.4580000340938568
111 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.4779999852180481
112 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.4039999842643738
113 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.5020000338554382
114 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.46799999475479126
115 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.5220000147819519
116 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.4660000205039978
117 hidden test final write donw!
FINAL R

208 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.4919999837875366
209 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.5119999647140503
210 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.5119999647140503
211 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.465999960899353
212 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.46399998664855957
213 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.46400001645088196
214 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.46000000834465027
215 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.46000000834465027
216 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.5140000581741333
217 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.4860000014305115
218 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.4880000352859497
219 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.49000000953674316
220 hidden test final write donw!
FI

FINAL RESULT: , Accuracy = 0.5040000081062317
312 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.4779999852180481
313 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.46000000834465027
314 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.4599999785423279
315 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.4580000042915344
316 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.46399998664855957
317 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.4440000057220459
318 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.47999998927116394
319 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.49000000953674316
320 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.4599999785423279
321 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.4620000123977661
322 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.49800002574920654
323 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.461999

415 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.5120000243186951
416 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.47600001096725464
417 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.5220000147819519
418 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.5299999713897705
419 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.46799999475479126
420 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.4739999771118164
421 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.484000027179718
422 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.49800002574920654
423 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.46400001645088196
424 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.48600003123283386
425 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.5199999809265137
426 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.5020000338554382
427 hidden test final write donw!
FI

FINAL RESULT: , Accuracy = 0.4959999918937683
519 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.465999960899353
520 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.5479999780654907
521 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.5180000066757202
522 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.5139999985694885
523 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.43799999356269836
524 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.4779999852180481
525 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.5139999389648438
526 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.5299999713897705
527 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.5
528 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.44999995827674866
529 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.5020000338554382
530 hidden test final write donw!
FINAL RESULT: , Accuracy = 0.4960000216960907
531 hidd